## Intro:
My client, Meagher + Geer, PLLP faced a data-centric issue on their case that required the cleaning, aggregating, and visualization of a large and complex set of data to help support their claim in a workers compensation suit. This was required for them to provide further context utilizing the quantitative data provided to them on their clients assumed working times over a multi-year period in the form of millions of raw time stamps and work functions performed by their client.

To tackle this issue, Meagher + Geer, PLLP hired Slater Bernstein consulting to perform the following key Analytical functions:

· Extract, transform and load the raw data into a normalized table containing the timestamps and work functions performed by their client over the outlined period. The raw tables came in the form of CSV files which were then extracted to a SQL schema.
    - *For confidentiality purposes, only column headers and dummy data will be displayed for all example visualizations.*

In [38]:

import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from IPython.display import display
import os
import csv

#Variables to be declared
host = os.getenv('host'),
user = os.getenv('user'),
password = os.getenv('pass'),
database = "HvT"

#Connection to MySQL for access FROM the database
try:
    mydb = mysql.connector.connect(
        host = os.getenv('host'),
        user = os.getenv('user'),
        password = os.getenv('pass'),
        database = "HvT"
    )
    curser = mydb.cursor()
    curser.execute("use HvT")

except mysql.connector.Error as err:
    print("An error occurred:", err)

#Connection to MySQL for access TO the database
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:3306/{database}")



curser.execute("SELECT `Control Number`, `Sort Date/Time - Offset` as DateTime, `date`, `time`, `DoW`, `Record Type` FROM Emails LIMIT 10;")
emails = curser.fetchall()
headers = [i[0] for i in curser.description]
emailsDf = pd.DataFrame(emails, columns=[headers], index=None)
display(emailsDf)

,Control Number,DateTime,date,time,DoW,Record Type
0,EML00149132,7/10/2018 11:42,7/10/2018,11:42:00 AM,Tuesday,Email
1,EML00149152,7/10/2018 13:59,7/10/2018,1:59:00 PM,Tuesday,Email
2,EML00162279,7/10/2018 15:58,7/10/2018,3:58:00 PM,Tuesday,Email
3,EML00162275,7/10/2018 16:01,7/10/2018,4:01:00 PM,Tuesday,Email
4,EML00162287,7/10/2018 16:02,7/10/2018,4:02:00 PM,Tuesday,Email
5,EML00162274,7/10/2018 16:02,7/10/2018,4:02:00 PM,Tuesday,Email
6,EML00162271,7/10/2018 16:09,7/10/2018,4:09:00 PM,Tuesday,Email
7,EML00162266,7/10/2018 16:17,7/10/2018,4:17:00 PM,Tuesday,Email
8,EML00162265,7/10/2018 16:20,7/10/2018,4:20:00 PM,Tuesday,Email
9,EML00162264,7/10/2018 16:21,7/10/2018,4:21:00 PM,Tuesday,Email
